In [1]:
# Necesario para interactuar y renderizar con ventana
import vtkmodules.vtkInteractionStyle # Si no se pone, no funciona interacción
import vtkmodules.vtkRenderingOpenGL2 # Si no se pone, no abre ventana


from vtkmodules.vtkCommonColor import vtkNamedColors

from vtkmodules.vtkFiltersSources import vtkConeSource

from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkLight,
    vtkCamera,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkRenderer
)

In [2]:
colors = vtkNamedColors()

In [3]:
coneSource = vtkConeSource()

mapper = vtkPolyDataMapper()
mapper.SetInputConnection(coneSource.GetOutputPort())

coneActor = vtkActor()
coneActor.SetMapper(mapper)

## Visualización

In [4]:
xGridDimensions = 3
yGridDimensions = 1

renderList = [vtkRenderer() for i in range(xGridDimensions*yGridDimensions)]
cameraList = [vtkCamera() for i in range(xGridDimensions*yGridDimensions)]
lightList = [vtkLight() for i in range(xGridDimensions*yGridDimensions)]

### Cámaras

In [5]:
pos = [[-3, 0, 3], [0, 0, 3], [3, 0, 3]]

for i in range(xGridDimensions*yGridDimensions):
    cameraList[i].SetClippingRange(0.1, 10)
    cameraList[i].SetFocalPoint(0,0,0)
    cameraList[i].SetPosition(*pos[i])
    cameraList[i].ComputeViewPlaneNormal()
    cameraList[i].SetViewUp(0,1,0)

### Luces

Vamos a situar las luces sobre las cámaras

In [6]:
for i in range(xGridDimensions*yGridDimensions):
    lightList[i].SetColor(colors.GetColor3d('White'))
    lightList[i].SetFocalPoint(cameraList[i].GetFocalPoint())
    lightList[i].SetPosition(cameraList[i].GetPosition())

### Renderers

In [7]:
viewport = [[0, 0, 0, 0] for i in range(xGridDimensions*yGridDimensions)]

for row in range(0, yGridDimensions):
    for col in range(0, xGridDimensions):
        # (xmin, ymin, xmax, ymax)
        index = row * xGridDimensions + col
        viewport[index][0] = col / xGridDimensions
        viewport[index][1] = (yGridDimensions - (row + 1)) / yGridDimensions
        viewport[index][2] = (col + 1) / xGridDimensions
        viewport[index][3] = (yGridDimensions - row) / yGridDimensions

for i in range(xGridDimensions*yGridDimensions):
    renderList[i].SetViewport(viewport[i])
    renderList[i].AddActor(coneActor)
    renderList[i].AddLight(lightList[i])
    renderList[i].SetActiveCamera(cameraList[i])

### Ventana de renderizado (no interactiva)

In [8]:
rendererSize = 256
renderWindow = vtkRenderWindow()

renderWindow.SetSize(rendererSize * xGridDimensions, rendererSize * yGridDimensions)
renderWindow.SetWindowName('Luces fijas con Cámaras')

for i in range(xGridDimensions*yGridDimensions):
    renderWindow.AddRenderer(renderList[i])
    
renderWindow.Render()

In [9]:
del renderWindow

Ahora vamos a hacer la cámara del segundo render fija para los otros dos

In [12]:
lightList[0].SetPosition(lightList[1].GetPosition())
lightList[2].SetPosition(lightList[1].GetPosition())

rendererSize = 256
renderWindow2 = vtkRenderWindow()

renderWindow2.SetSize(rendererSize * xGridDimensions, rendererSize * yGridDimensions)
renderWindow2.SetWindowName('Luz fija con segunda cámara')

for i in range(xGridDimensions*yGridDimensions):
    renderWindow2.AddRenderer(renderList[i])
    

renderWindow2.Render()

In [13]:
del renderWindow2